In [ ]:
using CSV
#using CurveFit
using Dates, DataFrames
#, Distributions, DSP
#using Gtk
#using LaTeXStrings
using NativeFileDialog
using Plots
using Printf
#using Tk

# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

# Select a Datawell daily .RDT file
infil = pick_file("C:\\", filterlist="*RDT");
##infil = "G:\\Wave_data\\Card Data\\mooloolaba\\Mooloolaba_WR_2018-2020\\07039ALO.RDT"
println("Selected ",infil)

#Change the type-interpretation of the binary file data to unsigned integer
println("Reading BINARY data from ",infil)
data_array = reinterpret(UInt8, read(infil));

date_time_list = []
north_hex_values = []

ii = 1
RDT_df = DataFrame([[],[],[],[],[],[]], ["Date", "UTC", "Heave", "North", "West", "GPS_error"])

# Convert df column types from 'Any' to their proper type
RDT_df.Date = map(DateTime, RDT_df.Date);
RDT_df.UTC = map(DateTime, RDT_df.UTC);
RDT_df.Heave = map(Float64, RDT_df.Heave);
RDT_df.North = map(Float64, RDT_df.North);
RDT_df.West = map(Float64, RDT_df.West);
RDT_df.GPS_error = map(Int32, RDT_df.GPS_error);
 

println("Decoding RDT data now")
flush(stdout)

while ii < length(data_array)
    
    start_of_message = string(data_array[ii], base=16, pad=2)
    message_id = string(data_array[ii+1], base=16, pad=2)
    message_length = parse(Int, string(data_array[ii+2], base=16, pad=2) * string(data_array[ii+3], base=16, pad=2), base= 16)
    check_sum1 = string(data_array[ii+4], base=16, pad=2)    

    yr = parse(Int,(string(data_array[ii+5], base=16) * string(data_array[ii+6], base=16, pad=2)), base= 16)

    month = parse(Int, string(data_array[ii+7], base=16, pad=2), base= 16)
    day = parse(Int, string(data_array[ii+8], base=16, pad=2), base= 16)
    hour = parse(Int, string(data_array[ii+9], base=16, pad=2), base= 16)
    minute = parse(Int, string(data_array[ii+10], base=16, pad=2), base= 16)

    # Calculate the sample rate
    sample_rate_hex = parse(UInt32,"0x"* string(data_array[ii+11], base=16, pad=2) * string(data_array[ii+12], base=16, pad=2) 
        * string(data_array[ii+13], base=16, pad=2) * string(data_array[ii+14], base=16, pad=2))
    sample_rate = reinterpret(Float32, parse(UInt32, "0x"*string(sample_rate_hex, base=16)))

    utc = DateTime(yr,month,day,hour,minute)
    aest = utc + Hour(10)
    push!(date_time_list,aest)

    if (sample_rate != 1.28f0) 

        println("Error: Sample rate not 1.28 Hz - Program terminated!")
        quit()

    end

#==
    println("Start of message = ",start_of_message)
    println("Message Id = ",message_id)
    println("Message length = ",message_length)
    println("Checksum = ",check_sum1)
    println("Date/Time (UTC) = ",utc)
    println("Sample rate = ",sample_rate," Hz.")
==#    
    rows = (message_length-10)/6
    
    for jj in 15:6:message_length
    
        # generate an array of dates at spacing equal to sample_rate
        aest = utc .+ Hour(10)

        heave = []
        north = []
        west = []

        # Calculate the displacements
        heave_hex = parse(UInt16,"0x"* string(data_array[ii+jj], base=16, pad=2) * string(data_array[ii+jj+1], base=16, pad=2))
        heave = reinterpret(Int16, parse(UInt16, "0x"*string(heave_hex, base=16))) / 100

        global north_hex = parse(UInt16,"0x"* string(data_array[ii+jj+2], base=16, pad=2) * string(data_array[ii+jj+3], base=16, pad=2))
        north = reinterpret(Int16, parse(UInt16, "0x"*string(north_hex, base=16))) / 100

        west_hex = parse(UInt16,"0x"* string(data_array[ii+jj+4], base=16, pad=2) * string(data_array[ii+jj+5], base=16, pad=2))
        west = reinterpret(Int16, parse(UInt16, "0x"*string(west_hex, base=16))) / 100

        push!(RDT_df,(utc .+ Hour(10), utc, heave, north, west, parse(Int, last(string(north_hex, base=2, pad=16),1))))
         
        # increment the record time
        utc = utc + Microsecond.(1/sample_rate * 1000000)

    end
    
    check_sum2 = string(data_array[message_length+1], base=16, pad=2)
    print(".")
    flush(stdout)
#==
    println("Checksum = ",check_sum2)
    println("_________________________________________")
==#
    
    ii += message_length + 6
    
end

# print number of GPS errors if they exist
gps_error_number = sum(RDT_df.GPS_error)
if gps_error_number > 0
    global gps_error_locations = findall(RDT_df.GPS_error .> 0)
    println("\nAlert: there were ",gps_error_number," errors in this record!\n")
    flush(stdout)
end

println("\nPreparing plot now...This takes a while!")
flush(stdout)

#==
# Plot all data over entire period
p1 = plot(RDT_df.Date,RDT_df.Heave, color=:blue, lw=:0.5, alpha=:0.5, title="Heave", label="")
if gps_error_number > 0
    heave_label = string(gps_error_number)*" GPS Errors"
    p1 = vline!(RDT_df.Date[gps_error_locations], alpha=:1, label=heave_label,lw=0.25, lc=:red)
end

p2 = plot(RDT_df.Date,RDT_df.North, color=:green, lw=:0.5, alpha=:0.5, title="North", label="")
p3 = plot(RDT_df.Date,RDT_df.West, color=:red, title="West", label="")

# create the date/time ticks for x-axis
tm_tick = range(first(RDT_df.UTC),last(RDT_df.Date),step=Hour(12))
ticks = Dates.format.(tm_tick,"dd/mm HH:MM")

rdt_plot = plot(p1, p2, p3, layout=(3,1), size=(1600,800),
    lw=:0.5,
    xlim=(first(RDT_df.UTC),last(RDT_df.Date)), xticks=(tm_tick,ticks), xtickfontsize=7,ytickfontsize=8, xminorticks=12,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, legend=:bottomleft,
    plot_title = Dates.format(date_time_list[1], "dd/mm/yy HH:MM")*" to "*Dates.format(date_time_list[end], "dd/mm/yy HH:MM"), plot_titlefontsize=:12, titlefontsize=:10,
    leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

flname = ".\\Plot_"*last(split(infil, "\\"))*".png"
savefig(flname)

display(rdt_plot)
==#

In [ ]:
start_record = DateTime(2009,05,10,6,0)
next_record = start_record + Minute(60)
jj = findall(start_record .<= RDT_df.Date .< next_record)
heave = RDT_df.Heave[jj]
timestamps = start_record .+ Microsecond.(collect(0:length(RDT_df.Heave[jj])-1) / 1.28 * 1000000)

tm_tick = range(first(timestamps),last(timestamps),step=Minute(5))
ticks = Dates.format.(tm_tick,"MM:SS")


plot(timestamps, heave, size=(1200,400), xlims=(timestamps[1],timestamps[end]), xticks=(tm_tick,ticks), ylims=(-2,2), framestyle = :box)

### Read all .RDT files in selected directory to df

In [ ]:
RDT_df.Date[jj[begin]]

In [ ]:
using CSV, CurveFit
using NativeFileDialog
using Glob
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf, Polynomials
using Statistics #, StatsPlotss
using StatsBase

function fix_gps_errors(heave, date, gps_flag)
# function to apply polynomial fit to WSE's affected by GPS errors
# uses selectable offset value to fine-tune result
#####################################

    gps_errors = findall(x -> x == 1, gps_flag)
    heave_length = length(heave)
    
    if !isempty(gps_errors)
        
        println(length(gps_errors)," GPS errors at ", Dates.format.(date, "yyyy-mm-dd HH:MM"))
        
        for ii ∈ reverse(gps_errors)

            error_center = ii

            # User-selected offset either side of GPS error
            lower_offset = upper_offset = 120

            if error_center <= lower_offset
                lower_offset = error_center - 1
            end

            if error_center+upper_offset > heave_length
                upper_offset = heave_length - error_center
            end

            # Ensure there are at least 3 points for fitting
            lower_offset = max(lower_offset, 2)
            upper_offset = max(upper_offset, 2)
    
            # Fit curve to subset of heave before GPS error
            left_side_points = error_center-lower_offset:error_center
            fit1 = curve_fit(Polynomial, left_side_points, heave[left_side_points], 2)
            yfit1 = fit1.(left_side_points)
            yfit1[length(yfit1)] = 0.0

            # Fit curve to subset of heave after GPS error
            right_side_points = error_center:error_center+upper_offset
            fit2 = curve_fit(Polynomial, right_side_points, heave[right_side_points], 2)
            yfit2 = fit2.(right_side_points)
            yfit2[1] = 0.0

            # apply polynomial results to wse's on both sides of GPS error
            heave[left_side_points] .= heave[left_side_points] - yfit1
            heave[right_side_points] .= heave[right_side_points] - yfit2
            heave[ii] = 0.0    # set wse at GPS error location to 0

        end
    
    end

    return(heave)
    
    end     # fix_gps_errors()


# smooth the spectra into bands centered on 0.05Hz spacing (i.e. 0:0.005:0.64)
function smooth_spectra(Pden_in, sample_frequency)
##################################################

    nyquist = sample_frequency/2

    freq_in = range(0, stop=nyquist, length=length(Pden_in))

    freq_out = [0.0]
    Pden_smoothed = [mean(Pden_in[1:8])]

    i = 9
    while i <= length(Pden_in)

        push!(freq_out,freq_in[i+8])

        if i < length(Pden_in)-16

            push!(Pden_smoothed, mean(Pden_in[i:i+16]))

        end

        i+=16

    end

    push!(Pden_smoothed, mean(Pden_in[end-8:end]))
            
    return (freq_out, Pden_smoothed)
        
end    # smooth_spectra()


function get_Fourier_coefficients(heave, north, west, sample_frequency)
#####################################################    

    # Get the cross periodograms
    cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
    cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
    cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

    cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
    cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
    cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

##    fhh = cps_heave_heave.freq
    fhh, Chh = smooth_spectra(real.(cps_heave_heave.power[1,1,:]), sample_frequency)

    #fnn = cps_north_north.freq
    fhh, Cnn = smooth_spectra(real.(cps_north_north.power[1,1,:]), sample_frequency)

    #fww = cps_west_west.freq
    fhh, Cww = smooth_spectra(real.(cps_west_west.power[1,1,:]), sample_frequency)

    #fnw = cps_north_west.freq
    fhh, Cnw = smooth_spectra(real.(cps_north_west.power[1,2,:]), sample_frequency)

    #fnh = cps_north_heave.freq
    fhh, Qnh = smooth_spectra(imag.(cps_north_heave.power[1,2,:]), sample_frequency)

    #fwh = cps_west_heave.freq
    fhh, Qwh = smooth_spectra(imag.(cps_west_heave.power[1,2,:]), sample_frequency)

    a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
    b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

    a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
    b2 = -2 .* Cnw ./ (Cnn .+ Cww)
    
    return(fhh, Chh, a1, b1, a2, b2)
    
end    # get_Fourier_coefficients()


function calc_f2_Pden2(heave, sample_frequency)
#############################
    
    # Show spectra using Welch's method to better define bimodal events
    ps_w = welch_pgram(heave, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning);
    f2 = freq(ps_w);
    Pden2 = power(ps_w)    

    return(f2, Pden2)

end    # calc_f2_Pden2()
    

function calc_spread_direction(a1,b1)
#####################################
    
    θ₀ = atan.(b1,a1)
    m1 = (a1.^2 .+ b1.^2).^0.5
##    m2 = a2.*cos.(2*θ₀) .+ b2.*sin.(2*θ₀)
##    n2 = -a2.*sin.(2*θ₀) .+ b2.*cos.(2*θ₀)

    # Calculate the spread
    σc = (2 .* (1 .- m1)).^0.5;

    direction = mod.(rad2deg.(atan.(b1,a1)),360)

    return(σc, direction)

end    # calc_spread_direction()


# Function to read binary file
function read_binary_file(filename)
###################################

    open(filename, "r") do file
        return(read(file))
    end

end    # read_binary_file()


# Function to find all header indices
function find_headers(data, header)
###################################

    header_indices = []
    header_length = length(header)
    data_length = length(data)
    
    for i in 1:(data_length - header_length + 1)
        if data[i:i+header_length-1] == header
            push!(header_indices, i)
        end
    end
    
    return(header_indices)

end    # find_headers()


#######################################################################################################
#######################################################################################################
#######################################################################################################

# Define the path to the directory you want to search in
directory_path = pick_folder()

# Use glob to find all .HXV files in the directory and subdirectories
rdt_files = glob(".//*.RDT", directory_path)
rdt_files = rdt_files[1:end-1]    # remove the file named TMP.RDT

# build df containing displacements and Fourier coefficients for selected day
global displacement_df = DataFrame(
    Date = DateTime[], 
    Heave = Vector{Float64}[], 
    North = Vector{Float64}[], 
    West = Vector{Float64}[], 
    fhh = Vector{Float64}[], 
    Chh = Vector{Float64}[], 
    a1 = Vector{Float64}[], 
    b1 = Vector{Float64}[], 
    a2 = Vector{Float64}[], 
    b2 = Vector{Float64}[], 
    f2 = Vector{Float64}[], 
    Pden2 = Vector{Float64}[], 
    Spread = Vector{Float64}[], 
    Direction = Vector{Float64}[]
)

for infil ∈ rdt_files

    println("Reading BINARY data from ",infil)
    data_array = reinterpret(UInt8, read(infil))
    
    date_time_list = []
    north_hex_values = []
    
    ii = 1
       
    while ii < length(data_array)
        
        start_of_message = string(data_array[ii], base=16, pad=2)
        message_id = string(data_array[ii+1], base=16, pad=2)
        message_length = parse(Int, string(data_array[ii+2], base=16, pad=2) * string(data_array[ii+3], base=16, pad=2), base= 16)
        check_sum1 = string(data_array[ii+4], base=16, pad=2)    
    
        yr = parse(Int,(string(data_array[ii+5], base=16) * string(data_array[ii+6], base=16, pad=2)), base= 16)
    
        month = parse(Int, string(data_array[ii+7], base=16, pad=2), base= 16)
        day = parse(Int, string(data_array[ii+8], base=16, pad=2), base= 16)
        hour = parse(Int, string(data_array[ii+9], base=16, pad=2), base= 16)
        minute = parse(Int, string(data_array[ii+10], base=16, pad=2), base= 16)
    
        # Calculate the sample rate
        sample_rate_hex = parse(UInt32,"0x"* string(data_array[ii+11], base=16, pad=2) * string(data_array[ii+12], base=16, pad=2) 
            * string(data_array[ii+13], base=16, pad=2) * string(data_array[ii+14], base=16, pad=2))
        sample_frequency = reinterpret(Float32, parse(UInt32, "0x"*string(sample_rate_hex, base=16)))
    
        utc = DateTime(yr,month,day,hour,minute)
        aest = utc + Hour(10)
        push!(date_time_list,aest)
    
        if (sample_frequency != 1.28f0) 
    
            println("Error: Sample rate not 1.28 Hz - Program terminated!")
            quit()
    
        end
    
    #==
        println("Start of message = ",start_of_message)
        println("Message Id = ",message_id)
        println("Message length = ",message_length)
        println("Checksum = ",check_sum1)
        println("Date/Time (UTC) = ",utc)
        println("Sample rate = ",sample_rate," Hz.")
    ==#    
        rows = (message_length-10)/6
        global heave = Float64[]; north = Float64[]; west = Float64[]; gps_flag = []
        
        for jj ∈ 15:6:message_length
        
            # generate an array of dates at spacing equal to sample_rate
            aest = utc .+ Hour(10)
       
            # Calculate the displacements
            heave_hex = parse(UInt16,"0x"* string(data_array[ii+jj], base=16, pad=2) * string(data_array[ii+jj+1], base=16, pad=2))
            push!(heave, reinterpret(Int16, parse(UInt16, "0x"*string(heave_hex, base=16))) / 100)
    
            north_hex = parse(UInt16,"0x"* string(data_array[ii+jj+2], base=16, pad=2) * string(data_array[ii+jj+3], base=16, pad=2))
            push!(north, reinterpret(Int16, parse(UInt16, "0x"*string(north_hex, base=16))) / 100)
    
            west_hex = parse(UInt16,"0x"* string(data_array[ii+jj+4], base=16, pad=2) * string(data_array[ii+jj+5], base=16, pad=2))
            push!(west, reinterpret(Int16, parse(UInt16, "0x"*string(west_hex, base=16))) / 100)
    
            # identify GPS error flags
            if parse(Int, last(string(north_hex, base=2, pad=16),1)) != 0
                push!(gps_flag,1)
            else
                push!(gps_flag,0)
            end

        end
        
        if sum(gps_flag) > 0
            heave = fix_gps_errors(heave, utc, gps_flag)
        end
    
        # get frequencies, spectra, and Fourier coefficients
        fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west, sample_frequency)
    
        f2, Pden2 = calc_f2_Pden2(heave, sample_frequency)
        σc, direction = calc_spread_direction(a1,b1)
    
        push!(displacement_df, (aest, heave, north, west, fhh, Chh, a1, b1, a2, b2, f2, Pden2, σc, direction))

        check_sum2 = string(data_array[message_length+1], base=16, pad=2)
        ii += message_length + 6
        
    end

end

# Sort the df by date ascending
sort!(displacement_df, :Date);

In [ ]:
sort!(displacement_df, :Date)
#Plots.plot(RDT_df.Date, RDT_df.Heave, label="", size=(1800,800))

### Use Seralize to save df to .bin file

In [ ]:
using CodecZlib, Serialization, DataFrames
using NativeFileDialog
using CSV
using Glob
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf
using Statistics #, StatsPlotss
using StatsBase

# Serialize the DataFrame to a file
outfil = ".\\Data\\" * split(directory_path,"\\")[end-1]*"_"*Dates.format(Date(Date(displacement_df.Date[1])), "yyyy-mm-dd")*
    "_to_"*Dates.format(Date(Date(displacement_df.Date[end])), "yyyy-mm-dd")*".bin"

println("Writing binary-formatted data to ",outfil)
flush(stdout)

@time begin

    open(outfil, "w") do io
        serialize(io, displacement_df)
    end

end

### Use Serialize to read gzipped .bin file to df

In [ ]:
using CodecZlib, Serialization, DataFrames
using NativeFileDialog
using CSV
using Glob
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf
using Statistics #, StatsPlotss
using StatsBase

##include("../Split_Spectra/Split_Spectra_Tools.jl")
##include("C://Users//PC1//Julia_programs//Split_Spectra//Split_Spectra_Tools.jl")

# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

const sample_frequency = 1.28

function read_gzip_file(io)
###########################
    
    gz = GzipDecompressorStream(io)                # Create a Gzip decompressor stream
    deserialized_displacement_df = deserialize(gz) # Deserialize the DataFrame from the decompressed stream
    close(gz)                                      # Close the decompressor stream
    
    return(deserialized_displacement_df)
    
end    # read_gzip_file()


#######################################################################################################
#######################################################################################################
#######################################################################################################


# Select the binary file
infil = pick_file("C:\\Users\\PC1\\Julia_programs\\Datawell\\Read_RDT\\Data\\", filterlist="*bin")

println("Selected ", infil)

@time begin
    
    # Deserialize the DataFrame from the file
    displacement_df = open(read_gzip_file, infil, "r")
    
end

# Verify the contents
##println(displacement_df)

println("Done!")

In [ ]:
### Use this for reading Noise_Floor

In [ ]:
infil = "C:\\Users\\PC1\\Julia_programs\\Datawell\\Read_RDT\\Data\\Noise_floor.bin"
println("Reading Noise Floor data from ",infil)
flush(stdout)

@time begin
    
    # Deserialize the DataFrame from the file
    noise_floors_df = open(read_gzip_file, infil, "r");
    
end

# Verify the contents
##println(noise_floors_df)

In [ ]:
using DataFrames
using Dates

# Step 1: Create the new DataFrame `records_df`
records_df = DataFrame(Date = DateTime[], Heave = Vector{Float64}[], North = Vector{Float64}[], West = Vector{Float64}[], GPS_error = Vector{Float64}[])

# Helper function to create 30-minute intervals
function create_intervals(df::DataFrame)
    intervals = []
    start_time = minimum(df.Date)
    end_time = maximum(df.Date)
    while start_time <= end_time
        push!(intervals, (start_time, start_time + Minute(30)))
        start_time += Minute(30)
    end
    return intervals
end

# Step 2: Iterate over the `may_2009_df` DataFrame, grouping data into 30-minute intervals
intervals = create_intervals(may_2009_df)

for (start_time, end_time) in intervals
    # Filter the data for the current 30-minute interval
    mask = (may_2009_df.Date .>= start_time) .& (may_2009_df.Date .< end_time)
    interval_df = may_2009_df[mask, :]

    if nrow(interval_df) > 0
        # Extract data for the columns
        heave_values = interval_df.Heave
        north_values = interval_df.North
        west_values = interval_df.West
        gps_errors_values = interval_df.GPS_error

        # Step 3: Populate the `records_df` DataFrame
        push!(records_df, (start_time, heave_values, north_values, west_values, gps_errors_values))
    end
end

# Display the first few rows to verify
first(records_df, 5)


In [ ]:
using Plots

p1 = Plots.plot(records_df.Heave[1])
p1 = Plots.plot!(records_df.North[1])

p1_plot = plot(p1, leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=true)



### Locate GPS errors in .RDT directories

In [ ]:
using CSV
using Dates, DataFrames
using NativeFileDialog
using Printf

error_df = DataFrame([[],[]],["Date", "GPS_errors"])

# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

rdt_directory = pick_folder()

# build list of all rdt files in selected directory
rdt_files = filter(x->occursin(".RDT",x), readdir(rdt_directory));
rdt_files = rdt_files[findall(x->endswith(uppercase(x), ".RDT"), rdt_files)];

for jj in rdt_files
    
    if jj != "TMP.RDT"
    
        infil = rdt_directory * "\\" * jj
    
    end
    
##    println(infil)
    print('.')
    
    data_array = reinterpret(UInt8, read(infil));

    ii = 1
    
    while ii < length(data_array)

        gps_error_sum = 0

        start_of_message = string(data_array[ii], base=16, pad=2)
        message_id = string(data_array[ii+1], base=16, pad=2)
        message_length = parse(Int, string(data_array[ii+2], base=16, pad=2) * string(data_array[ii+3], base=16, pad=2), base= 16)
        check_sum1 = string(data_array[ii+4], base=16, pad=2)    

        yr = parse(Int,(string(data_array[ii+5], base=16) * string(data_array[ii+6], base=16, pad=2)), base= 16)

        month = parse(Int, string(data_array[ii+7], base=16, pad=2), base= 16)
        day = parse(Int, string(data_array[ii+8], base=16, pad=2), base= 16)
        hour = parse(Int, string(data_array[ii+9], base=16, pad=2), base= 16)
        minute = parse(Int, string(data_array[ii+10], base=16, pad=2), base= 16)

        utc = DateTime(yr,month,day,hour,minute)
        aest = utc + Hour(10)

        for jj in 15:6:message_length

            gps_error_sum += parse(Int, last(string(parse(UInt16,"0x"* string(data_array[ii+jj+2], base=16, pad=2) * string(data_array[ii+jj+3], base=16, pad=2)), base=2, pad=16),1))
        end

        check_sum2 = string(data_array[message_length+1], base=16, pad=2)
        ii += message_length + 6

        if gps_error_sum > 0
#            println(aest,' ',gps_error_sum)
            push!(error_df,(aest,gps_error_sum))
        end

    end
    
end

# sort the df on dates        
error_df = sort(error_df, [order(:Date, rev=false)])


# create the date/time ticks for x-axis
tm_tick = range(first(error_df.Date), last(error_df.Date), step=Month(1))
ticks = Dates.format.(tm_tick,"mm-yy")

b1 = bar(error_df.Date, error_df.GPS_errors, lc=:blue, lw=:0.5, xrotation=90, size=(1800,500), 
    xlim=(first(error_df.Date),last(error_df.Date)), xticks=(tm_tick,ticks), xtickfontsize=7, ytickfontsize=12,
    ylim=(0,60), yminorticks=4, ylabel="No. errors",
    title="Daily GPS errors - " * (splitpath(rdt_directory))[end] * "\n\n" * Dates.format(first(error_df.Date), "dd/mm/yy") * " to " * Dates.format(last(error_df.Date), "dd/mm/yy"),
    leftmargin = 15Plots.mm, bottommargin = 25Plots.mm,
    grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, legend=false)

b1_plot = plot(b1)

flname = ".\\Plot_"*splitpath(rdt_directory)[end]

csv_file = flname*".csv"
plt_file = flname*".png"

CSV.write(csv_file, error_df)
savefig(plt_file)

display(b1_plot)

In [ ]:
SDT_noise

# **********************************************************************************************************************
## All below is Noise Floor code
# **********************************************************************************************************************

## Explore possibility of sorting RDT files by stat().mtime - unfortunately, only works with recent files

In [ ]:
using Dates, DataFrames
using NativeFileDialog
using Printf

# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

# select directory
rdt_directory = pick_folder()

# build list of all csv files in selected directory
rdt_files = filter(x->occursin(".RDT",x), readdir(rdt_directory));

a = []; b = []
for ii in (rdt_files)
    push!(a,ii)
    push!(b,Dates.unix2datetime.(mtime.(joinpath.(rdt_directory,ii))))
end

c = [a b]

c[sortperm(c[:,2]),:]

# Convert the array to a DataFrame
df = DataFrame(c, [:Event, :Date])

# Convert the Date column to Date type
#df.Date = Date.(df.Date, "yyyy-mm-dd HH:MM:SS")

# Sort the DataFrame by Date column
sorted_df = sort(df, :Date)

# Display the sorted DataFrame
println(sorted_df)

### extract the binary data from HEX files to df

In [ ]:
using Dates, DataFrames
using NativeFileDialog
using Printf

# Function to parse a date from hex values
function parse_date(data_array, index)
######################################    
    
    yr = parse(Int, string(data_array[index], base=16) * string(data_array[index + 1], base=16, pad=2), base=16)
    month = parse(Int, string(data_array[index + 2], base=16, pad=2), base=16)
    day = parse(Int, string(data_array[index + 3], base=16, pad=2), base=16)
    hour = parse(Int, string(data_array[index + 4], base=16, pad=2), base=16)
    minute = parse(Int, string(data_array[index + 5], base=16, pad=2), base=16)
    
    return(DateTime(yr, month, day, hour, minute))
    
end    # parse_date()


# Function to parse sample rate
function parse_sample_rate(data_array, index)
#############################################
    
    sample_rate_hex = parse(UInt32, "0x" * string(data_array[index], base=16, pad=2) *
        string(data_array[index + 1], base=16, pad=2) * string(data_array[index + 2], base=16, pad=2) *
        string(data_array[index + 3], base=16, pad=2))
    
    return(reinterpret(Float32, sample_rate_hex))

end    # parse_sample_rate()


# Function to parse displacement values
function parse_displacement(data_array, index)
##############################################
    
    hex_value = parse(UInt16, "0x" * string(data_array[index], base=16, pad=2) * string(data_array[index + 1], base=16, pad=2))
    
    return(reinterpret(Int16, hex_value) / 100)
    
end    # parse_displacement()

# Set the range of dates to be processed

# Hay Point MV Spartia long wave event
start_date = Date("01-05-2009", "dd-mm-yyyy") - Day(3)
end_date = Date("01-06-2009", "dd-mm-yyyy") + Day(3)

# Hay Point DWR-G buoy on shore - used to calculate Noise Floor
##start_date = Date("15-02-2018", "dd-mm-yyyy") - Day(3)
##end_date = Date("01-05-2018", "dd-mm-yyyy") + Day(3)

# QGHL DWR-G calibration
##start_date = Date("09-10-2023", "dd-mm-yyyy") - Day(3)
##end_date = Date("14-12-2024", "dd-mm-yyyy") + Day(3)

mask = (start_date .< sorted_df.Date .<= end_date)
infils = sorted_df[mask, :].Event

##infils=sorted_df.Event
# Create a DataFrame with specified column types
RDT_df = DataFrame(
    Date = DateTime[], 
    UTC = DateTime[], 
    Heave = Float64[], 
    North = Float64[], 
    West = Float64[], 
    GPS_error = Int[]
)

for infil in infils
    
    fil = rdt_directory * "\\" * infil
    println(fil)
    
    # Read the binary data from the file
    data_array = reinterpret(UInt8, read(fil))
    
    ii = 1
    
    while ii < length(data_array)
        
        message_length = parse(Int, string(data_array[ii + 2], base=16, pad=2) * string(data_array[ii + 3], base=16, pad=2), base=16)
        
        utc = parse_date(data_array, ii + 5)
        aest = utc + Hour(10)
        sample_rate = parse_sample_rate(data_array, ii + 11)
        
        if sample_rate != 1.28f0
            
            println("Error: Sample rate not 1.28 Hz - Program terminated!")
            quit()
            
        end
        
        for jj in 15:6:message_length
            
            heave = parse_displacement(data_array, ii + jj)
            north = parse_displacement(data_array, ii + jj + 2)
            gps_error = parse(UInt16, "0x" * string(data_array[ii + jj + 3], base=16, pad=2)) & 1
            west = parse_displacement(data_array, ii + jj + 4)
            
            push!(RDT_df, (aest, utc, heave, north, west, gps_error))
            utc += Microsecond(1 / sample_rate * 1e6)
            
        end
        
        ii += message_length + 6
        
    end
    
end

println("Done.")

### Convert sequential df to df of 30-minute rows

In [ ]:
# Create the new DataFrame `records_df`
records_df = DataFrame(Date = DateTime[], Heave = Vector{Float64}[], North = Vector{Float64}[], West = Vector{Float64}[], GPS_error = Vector{Float64}[])

# Helper function to create 30-minute intervals
function create_intervals(df::DataFrame)
########################################
    
    intervals = []
    # Set the range of dates to be processed
    start_time = DateTime.("01-05-2009 00:00", "dd-mm-yyyy HH:MM")
    end_time = DateTime.("01-06-2009 00:00", "dd-mm-yyyy HH:MM")

# Noise floor dates
##    start_time = DateTime("01-02-2018 00:00", "dd-mm-yyyy HH:MM")
##    end_time = DateTime("01-05-2018 00:00", "dd-mm-yyyy HH:MM")

##    start_time = DateTime("09-11-2023 00:00", "dd-mm-yyyy HH:MM")
##    end_time = DateTime("15-11-2023 00:00", "dd-mm-yyyy HH:MM")


    while start_time <= end_time
        push!(intervals, (start_time, start_time + Minute(30)))
        start_time += Minute(30)
    end
    
    return(intervals)
    
end    # create_intervals()


# Iterate over the DataFrame, grouping data into 30-minute intervals
intervals = create_intervals(RDT_df)

for (start_time, end_time) in intervals
    
    # Filter the data for the current 30-minute interval
    mask = (RDT_df.Date .>= start_time) .& (RDT_df.Date .< end_time)
    interval_df = RDT_df[mask, :]

    if nrow(interval_df) > 0
        
        # Extract data for the columns
        heave_values = interval_df.Heave
        north_values = interval_df.North
        west_values = interval_df.West
        gps_errors_values = interval_df.GPS_error

        # Step 3: Populate the `records_df` DataFrame
        push!(records_df, (start_time, heave_values, north_values, west_values, gps_errors_values))
        
    end
    
end

println("Done.")

In [ ]:
using CurveFit

# Function to check if there's at least one '1' in the array
function has_one(records_df::DataFrame, row_index::Int)
#######################################################
    
    return any(records_df.GPS_error[row_index] .== 1)
    
end    # has_one()
    

# function to apply polynomial fit to WSE's affected by GPS errors
function fix_gps_errors(index, df)  
# uses selectable offset value to fine-tune result
    
    # locate GPS errors
    global gps_errors = findall(x -> x == 1, df.GPS_error[index])
    
    if !isempty(gps_errors)
        
        heave = df.Heave[index]
        
        for ii ∈ reverse(gps_errors[3:end])

            error_center = ii

            # User-selected offset either side of GPS error
            lower_offset = upper_offset = 120

            if error_center <= lower_offset
                lower_offset = error_center - 1
            end

            if error_center+upper_offset > 2304
                upper_offset = 2304 - error_center
            end

            # Fit curve to subset of heave before GPS error
            left_side_points = error_center-lower_offset:error_center
            fit1 = curve_fit(Polynomial, left_side_points, heave[left_side_points], 2)
            yfit1 = fit1.(left_side_points)
            yfit1[length(yfit1)] = 0.0

            # Fit curve to subset of heave after GPS error
            right_side_points = error_center:error_center+upper_offset
            fit2 = curve_fit(Polynomial, right_side_points, heave[right_side_points], 2)
            yfit2 = fit2.(right_side_points)
            yfit2[1] = 0.0

            # apply polynomial results to wse's on both sides of GPS error
            heave[left_side_points] .= heave[left_side_points] - yfit1
            heave[right_side_points] .= heave[right_side_points] - yfit2
            heave[ii] = 0.0    # set wse at GPS error location to 0

        end
    
    end
    
    return(heave)
    
end     # fix_gps_errors()

records_df.Fixed_Heave = records_df.Heave

# Check each row in records_df
for row_index in 1:nrow(records_df)
    
    if has_one(records_df, row_index)
        
        records_df.Fixed_Heave[row_index] = fix_gps_errors(row_index, records_df)
        
    end
end

println("Done.")

### Calculate Spectra for each record

In [ ]:
using DSP
using DataFrames

# Sample frequency
Fs = 1.28

# Function to compute Welch's power spectral density
function compute_welch_psd(heave_data, Fs)
##########################################
    
    ps_w = welch_pgram(heave_data, 512, 256; onesided=true, nfft=512, fs=Fs, window=hanning)
    freqs = freq(ps_w)
    spectra = power(ps_w)
    
    return(freqs, spectra)
    
end    # compute_welch_psd()

# Preallocate arrays
num_rows = nrow(records_df)

f2 = Vector{Vector{Float64}}(undef, num_rows)
Pden2 = Vector{Vector{Float64}}(undef, num_rows)

# Iterate over records_df and compute PSD
for i in 1:num_rows
    
    print(".")
    heave_data = records_df.Heave[i]
    freqs, spectra = compute_welch_psd(heave_data, Fs)
    
    f2[i] = freqs
    Pden2[i] = spectra
    
end

println("Done!")

### Display the corrected spectra

### Do spectrogram of corrected spectra

In [ ]:
using Plots

gr()

display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

dates = records_df.Date
freqs = f2[1]
spectra = hcat(Pden2...)

max_spec = maximum(spectra)

start_date =  first(dates)
last_date = last(dates)

# display plots to screen
tm_tick = range(start_date,last_date,step=Day(1))
ticks = Dates.format.(tm_tick,"dd")

p1 = contourf(dates, freqs, spectra, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)
# draw grid lines on plot
for i in 0:0.1:0.6
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Day(1):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

#title = Dates.format(start_date, "yyyy-mm-dd") * " to " * Dates.format(last_date, "yyyy-mm-dd")
title1 = uppercase(split(rdt_directory,"\\")[end-1])*"_"*monthname(dates[1])*"_"*string(year(dates[1]))
plot_file = ".\\Plots\\"*title1*"_monthly_spectrogram.png"

p1_plot = plot(p1, xlabel="Date", xlim=(start_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.4), ytickfontsize=8, 
        title=title1, framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=true)

#==
try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end
==#
display(p1_plot)

In [ ]:
dates = records_df.Date

# Initialize an empty array to store daily intervals
dates_array = Date[]

# Iterate through dates and extract dates at daily intervals
for i in 1:length(dates)
    if i == 1 || day(dates[i]) != day(dates[i-1])
        push!(dates_array, dates[i])
    end
end

using Tk

dates_array = Dates.format.(dates_array, "yyyy-mm-dd")

w = Toplevel("Select Start Date", 235, 650)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates_array)

scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Tk.Button(f, "Ok")
pack(b)

println("Select a time from the menu!")
flush(stdout)

bind(b, "command") do path
                    
    file_choice = get_value(lb);

    global start_date = DateTime(file_choice[1])

end

println("Done.")

In [ ]:
using Plots

gr()

last_date = start_date + Day(5)
# display plots to screen
tm_tick = range(start_date,last_date,step=Hour(6))
ticks = Dates.format.(tm_tick,"dd HH:MM")

p5 = contourf(dates, freqs, spectra, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

# draw grid lines on plot
for i in 0:0.1:0.6
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Hour(6):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

title = title1*"_"*Dates.format(start_date, "yyyy-mm-dd HH:MM") * " to " * Dates.format(last_date, "yyyy-mm-dd HH:MM")
plt_file5 = ".\\Plots\\"*title
plt_file5 = replace(plt_file5, ":" => "")
plt_file5 = replace(plt_file5, " " => "_") * "_5day_specrtogram.png" 
    
p5_plot = plot(p5, xlabel="Date", xlim=(start_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.4), ytickfontsize=8, 
        title=title, framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=true)

try
                                                
    savefig(plt_file5)
    println("\nPlot file saved as "*plt_file5)

catch

    "Alert: Plot not saved!"

end

display(p5_plot)

In [ ]:
using StatsBase

RDT_noise = DataFrame(Dict("Frequency" => f2, "Spectra" => Pden2))

In [ ]:
using DataFrames
using Statistics
using FFTW

# Define a function to calculate MAD for a 1D array
function mad(arr)
    med = median(arr)
    mad_value = median(abs.(arr .- med))
    return mad_value
end

# Define a function to calculate MAD for a matrix
function mad_matrix(matrix; dims=1)
    if dims == 1
        return [mad(matrix[:, i]) for i in 1:size(matrix, 2)]
    elseif dims == 2
        return [mad(matrix[i, :]) for i in 1:size(matrix, 1)]
    else
        error("Unsupported dimension: $dims")
    end
end

# Extract all spectral arrays from the DataFrame
spectral_values = Pden2

# Convert the list of arrays into a matrix where each row is a spectrum
spectral_matrix = hcat(spectral_values...)'

# Calculate the mean spectra (mean of each column) directly
mean_spectra = mean(spectral_matrix, dims=1)
mean_spectra_vector = vec(mean_spectra)

# Calculate the median spectra (median of each column)
median_spectra = median(spectral_matrix, dims=1)

# Calculate the MAD (median absolute deviation) for each frequency
mad_spectra = mad_matrix(spectral_matrix, dims=1)

# Convert the results to vectors
median_spectra_vector = vec(median_spectra)
mad_spectra_vector = vec(mad_spectra)

println("Mean Spectra: ", mean_spectra_vector)
println("Median Spectra: ", median_spectra_vector)
println("MAD Spectra: ", mad_spectra_vector)

# Define a function to remove outliers using MAD
function remove_outliers_using_mad(matrix, threshold=3.0)
    median_vals = median(matrix, dims=1)
    mad_vals = mad_matrix(matrix, dims=1)
    lower_bound = median_vals .- threshold .* mad_vals
    upper_bound = median_vals .+ threshold .* mad_vals
    filtered_matrix = copy(matrix)
    
    for i in 1:size(matrix, 2)  # iterate over columns
        for j in 1:size(matrix, 1)  # iterate over rows
            if matrix[j, i] < lower_bound[i] || matrix[j, i] > upper_bound[i]
                filtered_matrix[j, i] = median_vals[i]
            end
        end
    end
    
    return filtered_matrix
end

# Remove outliers from the spectral matrix
filtered_spectral_matrix = remove_outliers_using_mad(spectral_matrix)

# Calculate the mean spectra after outlier removal
mean_spectra_filtered = mean(filtered_spectral_matrix, dims=1)
mean_spectra_vector_filtered = vec(mean_spectra_filtered)

println("Filtered Mean Spectra: ", mean_spectra_vector_filtered)

# Compute the difference spectra
difference_spectra = mean_spectra_vector - mean_spectra_vector_filtered

println("Difference Spectra: ", difference_spectra)

# Perform the inverse Fourier transform to get the time series of the outliers
time_series_outliers = real(ifft(difference_spectra))

println("Time Series of Outliers: ", time_series_outliers)


In [ ]:
using Plots

p1 = Plots.plot()
    
for ii in 1:length(f2)
    p1 = Plots.plot!(f2[ii], Pden2[ii], lc=:gray, alpha=:0.25, label="")
end
##==
for ii in 1:nrow(RDT_noise)
    p1 = Plots.plot!(RDT_noise.Frequency[ii], RDT_noise.Spectra[ii], lc=:lightgrey, alpha=:0.1, label="")
end
#==#
p1 = Plots.plot!(f2[1],  mean_spectra_vector, lw=:5, lc=:yellow, label="Mean")

p1 = Plots.plot!(f2[1],  median_spectra_vector, lw=:2, lc=:lightblue, label="Median")

##p1 = Plots.plot!(SDT_noise.Freq[1], mad_spectra_vector, lw=:2, lc=:pink, label="MAD")

p1 = Plots.plot!(f2[1], mean_spectra_vector_filtered, lw=:2, lc=:blue, label="Filtered")

p1_plot = plot(p1, layout=(1,1), size=(1800,800), lw=:0.5, fillrange = 0, fillalpha = 0.05, 
    xlim=(0.0,0.58), xtickfontsize=12, ytickfontsize=12, xminorticks=12, xlabel="Frequency (Hz)",
    yaxis=:log, ylim=(0.000001,10),  ylabel="S(f) (m²/Hz)", legend=:topright, legendfontpointsize=:12, legendtitlefonthalign=:right,
    legendtitle="Noise floor based on "*string(nrow(RDT_noise))*" spectra", legendtitlefontpointsize=:14,
    guidefontsize=:14, framestyle = :box, fg_legend=:transparent, bg_legend=:transparent, 
    title = Dates.format(records_df.Date[1], "dd/mm/yy HH:MM")*" to "*Dates.format(records_df.Date[end], "dd/mm/yy HH:MM"),
    titlefontsizes=:16, leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

display(p1_plot)

# **********************************************************************************************************************
## All below is work in progress
# **********************************************************************************************************************

## Investigate Cross Power Spectra

In [ ]:
## day1 = RDT_df.Date[1]
day1_string = Dates.format.(day1, "yyyy-mm-dd HH:MM")
day2 = RDT_df.Date[end]
day2_string = Dates.format.(day2 + Day(1), "yyyy-mm-dd HH:MM")

println("\nEnter START date <DD/MM/YYYY> between ",day1_string," and ",day2_string)
flush(stdout)
global start_date_str = readline()

println(start_date_str)
flush(stdout)

first_date = DateTime(start_date_str, "dd/mm/yyyy")

# do rough check that valid date has been entered
if (day1 < first_date ) & (first_date < day2) 
    
    # get data for selected day
    daily = RDT_df[findall(first_date .<= RDT_df.Date .< (first_date + Day(1))),:];

    start_time = daily.Date[1]
    dates = [(julian2datetime.(datetime2julian(start_time)) .+ Minute(i*30)) for i in collect(0:1:47)]
    date_string = Dates.format.(dates, "yyyy-mm-dd HH:MM:SS")

    using Tk

    w = Toplevel("Select Date", 235, 600)
    tcl("pack", "propagate", w, false)
    f = Frame(w)
    pack(f, expand=true, fill="both")

    f1 = Frame(f)
    lb = Treeview(f1, date_string)
    scrollbars_add(f1, lb)
    pack(f1,  expand=true, fill="both")

    tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
    b = Button(f, "Ok")
    pack(b)

    bind(b, "command") do path

        date_choice = get_value(lb)
        idx = indexin(date_choice, date_string)
        global idx = idx[1]

##        println(idx,' ',date_string[idx])

    end

    record = daily[dates[idx] .<= daily.Date .< dates[idx] + Minute(30),:];
    
else
    
    println("Invalid selection Try again!") 

end

title_string = Dates.format(record.Date[1], "yyyy-mm-dd HH:MM")
println(idx," ",record.Date[1]," ",title_string)
flush(stdout)

In [ ]:
RDT_df

In [ ]:
using DSP

npts = 2048
heave = record[1:npts,3]

sample_frequency = 1.28
nyquist = sample_frequency/2
n=div(length(heave), 9)

spec1 = periodogram(heave; onesided=eltype(heave)<:Real, nfft=nextfastfft(size(heave))[1], fs=1.28, window=hanning)
spec = welch_pgram(heave, n, div(n, 2); onesided=eltype(heave)<:Real, nfft=nextfastfft(n), fs=1.28, window=hanning)      

p1 = plot(freq(spec1), power(spec1), lc=:lightblue, lw=:2, z_order=:1, label="Periodogram")
p1 = plot!(freq(spec), power(spec), lc=:blue, lw=:2, z_order=:2, label="Welch's method\n")

#spec = mt_pgram(heave; onesided=eltype(heave)<:Real, nfft=nextfastfft(size(heave))[1], fs=1.28, nw=4, ntapers=ceil(2nw)-1, window=dpss(length(heave), nw, ntapers))

cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);

p1 = plot!(cps_heave_heave.freq, real.(cps_heave_heave.power[1,1,:]), lc=:red, lw=:2, fillto=:0, fillcolor=:red, fillalpha=:0.125, z_order=:3, label="Cross Power Spectra")
            
spectral_plot = plot(p1, size=(1200,600), title=title_string,
    xlim=(0,nyquist), xtickfontsize=7, xminorticks=5, ylim=(0,maximum(power(spec1)*1.05)), ytickfontsize=8, yminorticks=10,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
    leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)


## Calculate Fourier coefficients

In [ ]:
## Hint came from https://discourse.julialang.org/t/spectral-coherence-in-julia/106144/9 rafael.guerra
using DSP


function atan2(b,a)    
#########################
"""
    function to calculate direction from Fourier coefficients a1 and b1
    and return result in Radians
    
    Note: refer to https://en.wikipedia.org/wiki/Atan2
    
    Calls: Function atan2()
    Inputs: b and a 
    Returns: 0 <= c <= 2π
    
""" 
    len = length(b)
    c = zeros(len)
    
    for i in 1:length(b)

        # if both a1 and b1 are 0 then return 0 (to avoid NaN)
        if (a[i]!=0) & (b[i]!=0)

            c[i] = atan(b[i] / a[i])

            if a[i] >= 0

                if b[i] >= 0

                    c[i] = pi/2 - abs(c[i])

                else

                    c[i] = pi/2 + abs(c[i])

                end            

            else

                if b[i] >= 0

                    c[i] = 3*pi/2  + abs(c[i])

                else

                    c[i] = 3*pi/2 - abs(c[i])

                end

            end

        end
        
    end
    
    # return direction in Degrees
    return(c)
        
    end    # atan2()


function atan2d(b,a)
#########################
"""
    function to calculate direction from Fourier coefficients a1 and b1
    and return result in Degrees
    
    Note: refer to https://en.wikipedia.org/wiki/Atan2
    
    Calls: Function atan2()
    Inputs: b and a 
    Returns: 0 <= c <= 360
    
"""
    
    return(rad2deg.(atan2(b,a)))
            
end


sample_frequency = 1.28
nyquist = sample_frequency/2

npts = 2048 #2304

record = 1  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

y = npts*record
x = y-npts+1

heave = RDT_df[1:npts,3]
north = RDT_df[1:npts,4]
west = -RDT_df[1:npts,5]

# Get the cross periodograms
cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

fhh = cps_heave_heave.freq
Chh = real.(cps_heave_heave.power[1,1,:])

fnn = cps_north_north.freq
Cnn = real.(cps_north_north.power[1,1,:])

fww = cps_west_west.freq
Cww = real.(cps_west_west.power[1,1,:])

fnw = cps_north_west.freq
Cnw = real.(cps_north_west.power[1,2,:])

fnh = cps_north_heave.freq
Qnh = imag.(cps_north_heave.power[1,2,:])

fwh = cps_west_heave.freq
Qwh = imag.(cps_west_heave.power[1,2,:])

title_string = Dates.format(RDT_df[1,:].Date, "yyyy-mm-dd HH:MM:SS")

p1 =  plot(fhh, Chh, lc=:red, lw=:1, title="Chh")
p2 =  plot(fnn, Cnn, lc=:red, lw=:1, title="Cnn")
p3 =  plot(fww, Cww, lc=:red, lw=:1, title="Cww")
p4 =  plot(fnw, Cnw, lc=:red, lw=:1, title="Cnw")
p5 =  plot(fnh, Qnh, lc=:red, lw=:1, title="Qhn")
p6 =  plot(fwh, Qwh, lc=:red, lw=:1, title="Qhw")

cross_spectral_plot = plot(p1, p2, p3, p4, p5, p6, label="", size=(1600,600), layout=(2,3),
    lw=:0.5,
    xlim=(0,nyquist), xtickfontsize=7,ytickfontsize=8, xminorticks=5,
    plot_title=title_string, titlefontsize=:10,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
    topmargin = 0Plots.mm, leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

savefig(".\\" * "RDT_file_"*split(splitdir(infil)[end],".")[1] * "_Cross_spectra.png")

display(cross_spectral_plot)

a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
b2 = -2 .* Cnw ./ (Cnn .+ Cww)

p1 =  plot(fhh, a1, lc=:green, lw=:1, title="a1")
p2 =  plot(fhh, b1, lc=:green, lw=:1, title="b1")
p3 =  plot(fhh, a2, lc=:green, lw=:1, title="a2")
p4 =  plot(fhh, b2, lc=:green, lw=:1, title="b2")

###################################################
spectral_coefficients_plot = plot(p1, p2, p3, p4, label="", size=(1600,600), layout=(2,2),
    lw=:0.5,
    xlim=(0,nyquist), xtickfontsize=7,ytickfontsize=8, xminorticks=5,
    titlefontsize=:10,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
    leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

savefig(".\\"*"RDT_file_"*split(splitdir(infil)[end],".")[1]*"_Spectral_coefficients.png")

display(spectral_coefficients_plot)


theta = atan.(b1,a1)
m1 = (a1.^2 .+ b1.^2).^0.5
m2 = a2 .* cos.(2 .* theta) .+ b2 .* sin.(2 .* theta)
n2 = -a2 .* sin.(2 .* theta) .+ b2 .* cos.(2 .* theta)

p1 = plot(fhh, m1, lc=:blue, lw=:1, title="m1")
p2 = plot(fhh, m2, lc=:blue, lw=:1, title="m2")
p3 = plot(fhh, n2, lc=:blue, lw=:1, title="n2")

###################################################
centred_Fourier_coefficients_plot = plot(p1, p2, p3, label="", size=(1600,300), layout=(1,3),
    lw=:0.5,
    xlim=(0,nyquist), xtickfontsize=7,ytickfontsize=8, xminorticks=5,
    titlefontsize=:10,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
    leftmargin = 5Plots.mm, rightmargin = 0Plots.mm, bottommargin = 5Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

savefig(".\\"*"RDT_file_"*split(splitdir(infil)[end],".")[1]*"_centred_Fourier_coefficients.png")

display(centred_Fourier_coefficients_plot)

##dirn = mod.(rad2deg.(theta .- pi), 360)
dirn = mod.(atan2d(b1, a1) .- 90, 360)
spread = rad2deg.((2 .- 2 .* m1).^0.5)

w = spread

# plot the direction
using StatsBase

title_string = Dates.format(RDT_df[1,:].Date, "yyyy-mm-dd HH:MM:SS")*" AEST"

weighted_mean = mean(dirn,weights(Chh))
fp = fhh[argmax(Chh)]

p1 = plot(fhh, dirn, ribbon = w, lw=:1, ls=:dot, c=:blue, fillalpha = 0.25, yflip = false, tickfonthalign=:right,
        ylabel="Direction (ᵒ)", yguidefontcolor=:blue, yguidefontrotation=:180, ytickfontcolor=:blue, 
        yticks = 0:30:360, yminorgrid=:true, yminorticks=:3, ylim=(0,360), label="Direction", titlefontsize=10,
        framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, legend=:topleft, xlim=(0,0.64),
        grid=:true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, tickfontsize=7, z_order=:1)



p1 = hline!([weighted_mean], lc=:blue, ls=:dash, label="Avg. Direction = "*string(round.(weighted_mean; digits=1))*"ᵒ")
p1 = vline!([fp], lc=:red, ls=:dashdot, label="Tp = "*string(round.(1/fp; digits=1))*"s (Peak freq. = "*string(round.(fp; digits=2))*"Hz)")

# calculate and plot the spectra
cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency)

n=div(length(heave), 9)
spec = welch_pgram(heave, n, div(n, 2); onesided=eltype(heave)<:Real, nfft=nextfastfft(n), fs=1.28, window=hanning)

ymax_val = max(maximum(spec.power),maximum(real.(cps_heave_heave.power[1,1,:])))
p1 = plot!(twinx(), cps_heave_heave.freq, real.(cps_heave_heave.power[1,1,:]), lc=:red, lw=:1, fillrange = 0, fillcolor=:red, fillalpha = 0.25, label="Spectra", ylabel="Spectral Density (m²/Hz.)", 
        yguidefontcolor=:red, ytickfontcolor=:red, legend=:topright, xlim=(0,0.64), ylim=(0,ymax_val))

p1 = plot!(twinx(), freq(spec), power(spec), lc=:red, lw=:2, ls=:dot, label="\nWelch", xlim=(0,0.64), ylim=(0,ymax_val))

###################################################

direction_spectra_plot = plot(p1, size=(1600,800), xtickfontsize=7,ytickfontsize=8, xminorticks=5, title="Chh and Direction", titlefontsize=:10,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent,
    leftmargin = 15Plots.mm, rightmargin = 12Plots.mm, topmargin = 0Plots.mm)

savefig(".\\"*"RDT_file_"*split(splitdir(infil)[end],".")[1]*"_Chh_Dir.png")

display(direction_spectra_plot)

In [ ]:
plot(dirn)

In [ ]:
i = 1:aa
j = 1:1:361

f(i, j) = begin
    
    max(Chh[i] * (a1[i]*cosd(j-1) + b1[i]*sind(j-1) + a2[i]*cosd((2*j-1)) + b2[j]*sind(2*(j-1))),0)

end

X = repeat(reshape(i, 1, :), length(j), 1)
Y = repeat(j, 1, length(i))
Z = map(f, X, Y);


In [ ]:
#tm_tick = range(0,0.64,step=100)
ticks = 1:80:aa
ticklabels = [string(round(fhh[x], digits=2)) for x in ticks ]

p1 = contourf(i, j, f, lw=0.25, lc=:white, c=cgrad(:Spectral,rev=true), levels=50)
p1 = contour!(i, j, f, lc=:grey, lw=:0.5)
plot(p1, size=(800,800), xticks=(ticks,ticklabels),yticks=0:30:360)

In [ ]:
function smooth_spectra(Pden_in, sample_frequency)
##################################################
# smooth the spectra into bands centered on 0.05Hz spacing (i.e. 0:0.005:0.64)
    nyquist = sample_frequency/2

    freq_in = range(0, stop=nyquist, length=length(Pden_in))

    freq_out = [0.0]
    Pden_smoothed = [mean(Pden_in[1:8])]

    i = 9
    while i <= length(Pden_in)

        push!(freq_out,freq_in[i+8])

        if i < length(Pden_in)-16

            push!(Pden_smoothed, mean(Pden_in[i:i+16]))

        end

        i+=16

    end

    push!(Pden_smoothed, mean(Pden_in[end-8:end]))
            
    return (freq_out, Pden_smoothed)
        
end    # smooth_spectra()


function get_Fourier_coefficients(heave, north, west)
#####################################################    
    # Get the cross periodograms
    cps_heave_heave = mt_cross_power_spectra([heave heave]', fs=sample_frequency);
    cps_north_north = mt_cross_power_spectra([north north]', fs=sample_frequency);
    cps_west_west = mt_cross_power_spectra([west west]', fs=sample_frequency);

    cps_north_heave = mt_cross_power_spectra([north heave]', fs=sample_frequency);
    cps_west_heave = mt_cross_power_spectra([west heave]', fs=sample_frequency);
    cps_north_west = mt_cross_power_spectra([north west]', fs=sample_frequency);

##    fhh = cps_heave_heave.freq
    fhh, Chh = smooth_spectra(real.(cps_heave_heave.power[1,1,:]), sample_frequency)

    #fnn = cps_north_north.freq
    fhh, Cnn = smooth_spectra(real.(cps_north_north.power[1,1,:]), sample_frequency)

    #fww = cps_west_west.freq
    fhh, Cww = smooth_spectra(real.(cps_west_west.power[1,1,:]), sample_frequency)

    #fnw = cps_north_west.freq
    fhh, Cnw = smooth_spectra(real.(cps_north_west.power[1,2,:]), sample_frequency)

    #fnh = cps_north_heave.freq
    fhh, Qnh = smooth_spectra(imag.(cps_north_heave.power[1,2,:]), sample_frequency)

    #fwh = cps_west_heave.freq
    fhh, Qwh = smooth_spectra(imag.(cps_west_heave.power[1,2,:]), sample_frequency)

    a1 = Qnh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5
    b1 = -Qwh ./ ((Cnn .+ Cww) .* Chh) .^ 0.5

    a2 = (Cnn .- Cww) ./ (Cnn .+ Cww)
    b2 = -2 .* Cnw ./ (Cnn .+ Cww)
    
    return(fhh, Chh, a1, b1, a2, b2)
    
end    # get_Fourier_coefficients()


function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
end     # get_displacements()


function get_HNW(infil)
#####################################
        
    global df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));

    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    global sequence = []

    for i in arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    return(heave, north, west)

    end    # get_HNW()


function get_spec_dir(displacement_df, total)

    Chh = displacement_df.Chh[total]
    a1 = displacement_df.a1[total]
    b1 = displacement_df.b1[total] 
    a2 = displacement_df.a2[total] 
    b2 = displacement_df.b2[total]
    time_string[] = displacement_df.Time_string[total]

    aa = length(Chh) # Number of spectral points

    r = 1:3:aa
    ρ = r ./ (aa/nyquist) 

    θ = 0:pi/180:2pi

#    mat =  []

    mat = [Chh[j] * (a1[j]*cos(i) + b1[j]*sin(i) + a2[j]*cos(2i) + b2[j]*sin(2i)) for i in θ, j in r]

    mat[mat .< 0] .= 0
    
    return(θ, ρ, mat, time_string)
    
end    # get_spec_dir()

using CairoMakie
using DSP
using GLMakie
using Statistics

using CSV
#using CurveFit
using Dates, DataFrames
#, Distributions, DSP
#using Gtk
#using LaTeXStrings
using NativeFileDialog
using Plots
using Printf
#using Tk

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Select a Datawell daily .RDT file
infil = pick_file("C:\\", filterlist="*RDT");
##infil = "G:\\Wave_data\\Card Data\\mooloolaba\\Mooloolaba_WR_2018-2020\\07039ALO.RDT"
println("Selected ",infil)

#Change the type-interpretation of the binary file data to unsigned integer
println("Reading BINARY data from ",infil)
data_array = reinterpret(UInt8, read(infil));

date_time_list = []
north_hex_values = []

ii = 1
RDT_df = DataFrame([[],[],[],[],[],[]], ["Date", "UTC", "Heave", "North", "West", "GPS_error"])

# Convert df column types from 'Any' to their proper type
RDT_df.Date = map(DateTime, RDT_df.Date);
RDT_df.UTC = map(DateTime, RDT_df.UTC);
RDT_df.Heave = map(Float64, RDT_df.Heave);
RDT_df.North = map(Float64, RDT_df.North);
RDT_df.West = map(Float64, RDT_df.West);
RDT_df.GPS_error = map(Int32, RDT_df.GPS_error);
 

println("Decoding RDT data now")
flush(stdout)

while ii < length(data_array)
    
    start_of_message = string(data_array[ii], base=16, pad=2)
    message_id = string(data_array[ii+1], base=16, pad=2)
    message_length = parse(Int, string(data_array[ii+2], base=16, pad=2) * string(data_array[ii+3], base=16, pad=2), base= 16)
    check_sum1 = string(data_array[ii+4], base=16, pad=2)    

    yr = parse(Int,(string(data_array[ii+5], base=16) * string(data_array[ii+6], base=16, pad=2)), base= 16)

    month = parse(Int, string(data_array[ii+7], base=16, pad=2), base= 16)
    day = parse(Int, string(data_array[ii+8], base=16, pad=2), base= 16)
    hour = parse(Int, string(data_array[ii+9], base=16, pad=2), base= 16)
    minute = parse(Int, string(data_array[ii+10], base=16, pad=2), base= 16)

    # Calculate the sample rate
    sample_rate_hex = parse(UInt32,"0x"* string(data_array[ii+11], base=16, pad=2) * string(data_array[ii+12], base=16, pad=2) 
        * string(data_array[ii+13], base=16, pad=2) * string(data_array[ii+14], base=16, pad=2))
    sample_rate = reinterpret(Float32, parse(UInt32, "0x"*string(sample_rate_hex, base=16)))

    utc = DateTime(yr,month,day,hour,minute)
    aest = utc + Hour(10)
    push!(date_time_list,aest)

    if (sample_rate != 1.28f0) 

        println("Error: Sample rate not 1.28 Hz - Program terminated!")
        quit()

    end

    rows = (message_length-10)/6
    
    for jj in 15:6:message_length
    
        # generate an array of dates at spacing equal to sample_rate
        aest = utc .+ Hour(10)

        heave = []
        north = []
        west = []

        # Calculate the displacements
        heave_hex = parse(UInt16,"0x"* string(data_array[ii+jj], base=16, pad=2) * string(data_array[ii+jj+1], base=16, pad=2))
        heave = reinterpret(Int16, parse(UInt16, "0x"*string(heave_hex, base=16))) / 100

        global north_hex = parse(UInt16,"0x"* string(data_array[ii+jj+2], base=16, pad=2) * string(data_array[ii+jj+3], base=16, pad=2))
        north = reinterpret(Int16, parse(UInt16, "0x"*string(north_hex, base=16))) / 100

        west_hex = parse(UInt16,"0x"* string(data_array[ii+jj+4], base=16, pad=2) * string(data_array[ii+jj+5], base=16, pad=2))
        west = reinterpret(Int16, parse(UInt16, "0x"*string(west_hex, base=16))) / 100

        push!(RDT_df,(utc .+ Hour(10), utc, heave, north, west, parse(Int, last(string(north_hex, base=2, pad=16),1))))
         
        # increment the record time
        utc = utc + Microsecond.(1/sample_rate * 1000000)

    end
    
    check_sum2 = string(data_array[message_length+1], base=16, pad=2)
    print(".")
    flush(stdout)
#==
    println("Checksum = ",check_sum2)
    println("_________________________________________")
==#
    
    ii += message_length + 6
    
end

# print number of GPS errors if they exist
gps_error_number = sum(RDT_df.GPS_error)
if gps_error_number > 0
    global gps_error_locations = findall(RDT_df.GPS_error .> 0)
    println("\nAlert: there were ",gps_error_number," errors in this record!\n")
    flush(stdout)
end

const sample_frequency = 1.28
nyquist = sample_frequency/2

# Build daily df containing spectral data
record = 1
start_val = DateTime(Date(RDT_df.Date[1]))
end_val = start_val + Minute(30)
    
# build df containing displacements and Fourier coefficient for selected day
displacement_df = DataFrame(Time_string = [], Heave = [], North = [], West = [], fhh = [], Chh = [], a1 = [], b1 = [], a2 = [], b2 = [], mat = [])

println("\nProcessing each 30-minute record:")

while record <= (round((RDT_df.Date[end] - RDT_df.Date[1]).value * 0.001 / 1800))
        
    if (mod(record,10) == 0)
        print(string(record))
    else
        print(".")
    end
    
#    try
        # find all records for a 30-minute record
        temp_df = RDT_df[start_val .<= RDT_df.UTC .< end_val,:]
        heave, north, west = temp_df.Heave, temp_df.North, temp_df.West
            
        # calculate Fourier coefficients
        fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(temp_df.Heave, temp_df.North, temp_df.West)
        
        aa = length(Chh) # Number of spectral points
        
        r = 1:6:aa
        global ρ = r ./ (aa/nyquist) 

        global θ = 0:pi/180:2pi        

        # populate a matrix of spectral surface values
        mat = [Chh[r1] * (a1[r1]*cos.(θ) + b1[r1]*sin.(θ) + a2[r1]*cos.(2θ) + b2[r1]*sin.(2θ)) for r1 in r]
                    
        mat = hvcat(size(mat,1),mat...)

        # set any values less than zero to zero
        mat[mat .< 0] .= 0
        
        time_string = Dates.format(start_val, "dd/mm/yyyy HH:MM")

        # add spectral data to plot df
        push!(displacement_df, (time_string, heave, north, west, fhh, Chh, a1, b1, a2, b2, mat));
            
        record += 1
        start_val = end_val
        end_val += Minute(30)
#==
    catch
        
        println("Error")
        break
        
    end
==#    
end

println("\nPlotting 30-minute records now")
flush(stdout)

# get the highest energy value for the day
# this sets scaling of plots                    
spec_max = maximum(maximum.(displacement_df.mat))
                    
println("Maximum spectra for day = ",round(spec_max, digits=2),"m²/Hz.")
                    
# declare the Observables
inc = Observable(1)
time_string = Observable(displacement_df[inc[],:].Time_string)
mat = Observable(Float64.(displacement_df[inc[],:].mat))
fhh_L = Observable(Int(round(length(displacement_df.Chh[1]) / 6)))
        
fig = CairoMakie.Figure(size=(800, 850))

# draw the polar axis
ax = CairoMakie.PolarAxis(fig[1, 1],
    thetaticklabelsize = 15,  
    rlimits=(0,0.6), rticklabelsize=15, rticks=0:0.2:0.6, rgridwidth=0.5, rtickangle=180, rminorgridvisible=true, rminorgridstyle=:dot,
    theta_0=-pi/2, thetagridwidth=0.5, thetaminorgridvisible=true, thetaminorgridstyle=:dot, thetaminorticks=IntervalsBetween(3), 
    direction=-1, width=630, height=610, title=time_string, titlesize=24,
    )

# Set plotting values
cmap = Reverse(:ocean)
levels = round(spec_max/100, digits=2):round(spec_max/20, digits=2):round(spec_max, digits=2)
θ = 0:pi/180:2pi
ρ = range(0, stop=1.28, length=fhh_L[])

# do contour plot
c1 = CairoMakie.contourf!(ax, θ, ρ, mat, colormap=cmap, levels=levels)
c1 = CairoMakie.contour!(ax, θ, ρ, mat, colormap=cmap, levels=levels)
                    
ax = CairoMakie.Colorbar(fig[2, 1], limits=(0, round(spec_max, digits=1, RoundUp)), label="Spectral Density (m²/Hz.)", labelsize=:20, 
            width=500, height=30, vertical=false, flipaxis=false, colormap=cmap)  
display(fig) 

# update the Observables
for i in 1:nrow(displacement_df)
    
    inc = i
    time_string[] = displacement_df[inc[],:].Time_string
    
    try
        mat[] = Float64.(displacement_df.mat[inc[]])
    catch
        break
    end
    
    sleep(0.3)
    yield()
    
    
end

In [ ]:
println("\007")

In [ ]:
using WAV

y, fs = wavread("foo.wav")

wavplay(y, fs)